In [1]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import spacy
# import wordcloud
# import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')
# from collections import Counter

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import os
abspath = os.path.abspath('D:/Box Sync/editorials/scripts/')
os.chdir(abspath)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 150)

print("done")

done


In [2]:
import import_func as imp
import desc_vis as vis
import tags_work as tgs
# import D:/Box Sync/editorials/scripts/import_func as imp
# import D:/Box Sync/editorials/scripts/desc_vis as vis

# Data
elist = "D:/Box Sync/editorials/data/editorials-1.jl"
tags = "D:/Box Sync/editorials/data/by_article_110219.jl"

print("done")

done


Import and process the data from .jl files using import_func functions

In [3]:
elist = "../data/editorials-1.jl"
tags = "../data/by_article_110219.jl"

editorial = imp.import_jl(elist)
tags = imp.import_jl(tags)

out_edi = imp.process(editorial, "editorial")
out_tag = imp.process(tags, "tags")

edi_df = out_edi[0]
edi_dict = out_edi[1]

tag_df = out_tag[0]
tag_dict = out_tag[1]

edi_df.head()

,headline,preview,authors,date
6139,how to write a winning résumé,our columnist explains all you need to know about resumes and cvs,peter fiske,1996-10-18
6135,the commandments of cover letter creation,writing a good cover letter boils down to making a decent presentation of your experience with t...,peter fiske,1996-12-20
6134,his mother cried when he went into sales,"how one man's career change helped him keep his self-respect, make a few bucks, and have a blast...",david g. jensen,1997-05-09
6133,what are headhunters and how do they work?,do you know how the headhunting process works and how it might affect you someday?,david g. jensen,1997-09-12
6132,dressing scientists for success: male case study,"scientists may find shopping for business attire perplexing at first, but this is easily overcom...",peter fiske,1997-09-26


Clean the dataframes
* make a sequential date column for both dates and months
* indicate whether posts are columns or not (because early posts do not have a column tag, posts by authors that appear more than [5] times will be considered columns

In [4]:
edi_df = imp.seq_dates(edi_df, "editorial")
tag_df = imp.seq_dates(tag_df, "tags")

tags_df = imp.id_columns(tag_df)

In [5]:
tags_df = tgs.seperate_tags(tags_df)
tags_df.head()

,headline,authors,date,time,date_seq,month_seq,year,n_posts_author,column1,column2,variable,tag
0,how to write a winning résumé,peter fiske,1996-10-18,8:00 am,18,10,1996,59,no,yes,0,tooling up
1,the commandments of cover letter creation,peter fiske,1996-12-20,0:00 am,81,12,1996,59,no,yes,0,tooling up
2,dressing scientists for success: male case study,peter fiske,1997-09-26,8:00 am,361,21,1997,59,no,yes,0,tooling up
3,the tooling up book club: on the market,peter fiske,1998-01-30,0:00 am,487,25,1998,59,no,yes,0,tooling up
4,self-assessment exercises: a gre for your ego and superego,peter fiske,1998-02-27,0:00 am,515,26,1998,59,no,yes,0,tooling up


## Tags work ##

In [17]:
# Now I'm going to develop the tag_incidence function here
# because text_analysis1 has become unweildy
inc_dict = {}
problem_rows = [] #[6175]

# structure of inc_dict
# {tag: 
#     {"overall":
#         "total": int,
#         "span": [[date_seq,month_seq] [date_seq, month_seq],...]
#     }
#     {"author1": int, "author2": int, ... }
# }

lifespan = True
# print(len(tags_df))
# print(tags_df.loc[6175])


for i in range(1,6174):
    if i in problem_rows:
        continue
    # print(tags_df.loc[i, "authors"]," ", tags_df.loc[i, "tag"])
    w_tag = tags_df.loc[i, "tag"]
    w_author = tags_df.loc[i, "authors"]
    if w_tag in inc_dict.keys():
        inc_dict[w_tag]["overall"]["total"] += 1
        if w_author in inc_dict[w_tag].keys():
            inc_dict[w_tag][w_author] += 1
        else:
            inc_dict[w_tag][w_author] = 1
    else:
        inc_dict[w_tag] = {}
        inc_dict[w_tag]["overall"] = {}
        inc_dict[w_tag]["overall"]["total"] = 1
        if lifespan == True:
            inc_dict[w_tag][w_author] = 1
            inc_dict[w_tag]["overall"]["span"] = []
        else:
            inc_dict[w_tag][w_author] = 1
    if lifespan == True:
        inc_dict[w_tag]["overall"]["span"].append([tags_df.loc[i, "date_seq"],tags_df.loc[i, "month_seq"]])
        # print(tags_df.loc[i, "headline"], "###",tags_df.loc[i, "tag"], "###", tags_df.loc[i, "date_seq"] )
        # test = "test"

# count = 0
# while count < 100:
#     for key in inc_dict.keys():
#         print(key, inc_dict[key])
#         count +=1

# determine how likely it is for the tag to be specific to a particular column


In [49]:
# identify when it's likely that the tags are associated with columns
col_tag = {}

for key in inc_dict.keys():
    tot = inc_dict[key]["overall"]["total"]
    prop_author = []
    for author in inc_dict[key].keys():
        if author == "overall":
            continue
        else:
            prop_author.append(inc_dict[key][author]/tot)
    col_tag[key] = [inc_dict[key]["overall"]["total"]]
    col_tag[key].append(prop_author)

# print(len(inc_dict))
# print(len(col_tag))

#for key in col_tag.keys():
#    print(key, col_tag[key])
# print("career-related policy",len(col_tag["career-related policy"][1]), col_tag["career-related policy"])

##  Possible strategies
# max(prop of author) - (number of authors/incidence) = diff
# ----> if diff is higher, it's more likely to be a column tag for a

diff_set = []

for key in col_tag.keys():
    if len(col_tag[key][1]) == 1:
        diff = max(col_tag[key][1]) - len(col_tag[key][1])/col_tag[key][0]
    elif len(col_tag[key][1]) < 4:
        diff = max(col_tag[key][1]) - len(col_tag[key][1])/col_tag[key][0]
    else:
        diff = max(col_tag[key][1])
    col_tag[key].append(diff)
    diff_set.append(diff)

q = np.quantile(diff_set, .2)

for key in col_tag.keys():
    if col_tag[key][2] < q:
        print(key, col_tag[key][0], len(col_tag[key][1]), col_tag[key][2])


forensics 4 3 -0.25
communicating science 2 2 -0.5
europe 3 3 -0.6666666666666667
erc 2 2 -0.5
government shutdown 2 2 -0.5
computer science 3 3 -0.6666666666666667
olympic 2 2 -0.5
breakthrough of the year 2 2 -0.5
faculty job 3 3 -0.6666666666666667
sport 2 2 -0.5
esof 2014 2 2 -0.5
career advice 2 2 -0.5
entrepreneurship 3 3 -0.6666666666666667
archaeology 3 3 -0.6666666666666667
audacity 3 3 -0.6666666666666667
erc grant 2 2 -0.5
writing 2 2 -0.5
scientific publishing 3 3 -0.6666666666666667
mental health 4 3 -0.25
alternative career 2 2 -0.5
data sharing 3 3 -0.6666666666666667
networking 2 2 -0.5
mentoring 2 2 -0.5
twitter 2 2 -0.5
2014moneyissue 3 3 -0.6666666666666667
stem 2 2 -0.5
unionization 3 3 -0.6666666666666667
entrepreneur 2 2 -0.5
collaborators 2 2 -0.5
ebola 2 2 -0.5
impostor syndrome 2 2 -0.5
ace 4 3 -0.25
physician-scientist 2 2 -0.5


Descriptive for tags
* how many tags total
* average number of tags per post


In [47]:
# total tags
len(inc_dict)


189

# Publication Rates

In [ ]:
# Density of all posts by Sequential date
sns.distplot(editorial["date_seq"])

In [ ]:
# Posts each month
